In [1]:
# Treminologies

In [2]:
# FICO score:
'''FICO score is a credit score created by the Fair Isaac Corporation (FICO).
Lenders use borrowers' FICO scores along with other details on borrowers'
A FICO Score above 670 is better than average, while scores above 740 indicate 
that the borrower is extremely responsible with credit.
A FICO Score above 800 is considered exceptional.'''

"FICO score is a credit score created by the Fair Isaac Corporation (FICO).\nLenders use borrowers' FICO scores along with other details on borrowers'\nA FICO Score above 670 is better than average, while scores above 740 indicate \nthat the borrower is extremely responsible with credit.\nA FICO Score above 800 is considered exceptional."

In [3]:
# Hardship
'''You are in financial hardship if you have difficulty paying your bills and repayments
on your loans and debts when they are due.'''

'You are in financial hardship if you have difficulty paying your bills and repayments\non your loans and debts when they are due.'

In [4]:
# Settlement:
'''Settlement is the "final step in the transfer of ownership involving the physical
exchange of securities or payment". 
After settlement, the obligations of all the parties have been discharged and 
the transaction is considered complete.'''

'''Exchange of payment to the seller and 
the Transfer of securities to the buyer of a trade.'''

'Exchange of payment to the seller and \nthe Transfer of securities to the buyer of a trade.'

# PROBLEM CATEGORY

#### RISK ANALYSIS

# 1. PROBLEM STATEMENT

#### TO MINIMIZE THE RISK MUDDLED IN FINANCIAL INVESTMENTS

## IMPORTING REQUIRED LIBRARIES

In [5]:
# Installing Catboost 
# CatBoost is an open-source software library developed by Yandex.
# It provides a gradient boosting framework which among other features attempts to
# solve for Categorical features using a permutation driven alternative compared to 
# the classical algorithm

In [6]:
# !pip install catboost==0.15 # Downgraded to the previous version due to the critical bug in 0.16

In [2]:
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from catboost import Pool, CatBoostClassifier

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

import seaborn as sns
import matplotlib.pyplot as plt

## 2. DATA GATHERING 

In [63]:
df_acc = pd.read_csv('accepted.csv')

/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1293/1274077888.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acc = pd.read_csv('accepted.csv')


In [9]:
df_num = df_acc.select_dtypes('number')
df_obj = df_acc.select_dtypes('object')

In [10]:
df_acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,88985880,NaN,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260697,88224441,NaN,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,...,NaN,NaN,Cash,Y,Mar-2019,ACTIVE,Mar-2019,10000.0,44.82,1.0
2260698,88215728,NaN,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
col_list = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens',
       'hardship_flag', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'fico_range_high', 'fico_range_low',
       'last_fico_range_high', 'last_fico_range_low', 'tot_coll_amt',
       'tot_cur_bal', 'verification_status_joint']
# print(col_list)
# print(len(col_list))

In [24]:
# col_list_1 = ['num_tl_120dpd_2m', 'id','url', 'emp_title','grade', 'int_rate',
# 'earliest_cr_line','sec_app_earliest_cr_line','emp_length','desc', 'member_id', 'verification_status_joint','bc_open_to_buy', 'mo_sin_old_il_acct', 
# 'mths_since_last_delinq','mths_since_last_major_derog', 'mths_since_last_record',
# 'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
# 'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
# 'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']
# len(col_list_1)

In [38]:
col_list_1_new = ['num_tl_120dpd_2m', 'id', 'url', 'sec_app_earliest_cr_line', 'member_id', 'bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'pct_tl_nvr_dlq', 'sec_app_mths_since_last_major_derog']
len(col_list_1_new)

17

In [6]:
# np.intersect1d(col_list_1,col_list)

array(['desc', 'earliest_cr_line', 'emp_length', 'emp_title', 'grade',
       'int_rate', 'verification_status_joint'], dtype='<U35')

In [32]:
np.intersect1d(col_list_1_new,col_list)

array([], dtype='<U35')

In [49]:
# col_list_final

In [61]:
print(len(col_list_1_new))
print(len(col_list))
print(len(col_list_final))

17
61
78


In [58]:
col_list_final = ['num_tl_120dpd_2m', 'id', 'url', 'sec_app_earliest_cr_line',
       'member_id', 'bc_open_to_buy', 'mo_sin_old_il_acct',
       'mths_since_last_delinq', 'mths_since_last_major_derog',
       'mths_since_last_record', 'mths_since_rcnt_il',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'pct_tl_nvr_dlq', 'sec_app_mths_since_last_major_derog',
       'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens',
       'hardship_flag', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'fico_range_high', 'fico_range_low',
       'last_fico_range_high', 'last_fico_range_low', 'tot_coll_amt',
       'tot_cur_bal', 'verification_status_joint']

In [67]:
df1 = df_acc[col_list_final]

In [68]:
df_csv = df1.sample(100000,ignore_index=True)
df_csv

,num_tl_120dpd_2m,id,url,sec_app_earliest_cr_line,member_id,bc_open_to_buy,mo_sin_old_il_acct,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,...,hardship_flag,debt_settlement_flag,debt_settlement_flag_date,fico_range_high,fico_range_low,last_fico_range_high,last_fico_range_low,tot_coll_amt,tot_cur_bal,verification_status_joint
0,0.0,89824638,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,8248.0,156.0,16.0,NaN,79.0,...,N,N,NaN,689.0,685.0,574.0,570.0,0.0,351251.0,NaN
1,0.0,135727227,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,36119.0,187.0,5.0,19.0,NaN,...,N,N,NaN,704.0,700.0,719.0,715.0,958.0,98611.0,NaN
2,0.0,104347194,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,4463.0,155.0,15.0,NaN,115.0,...,N,N,NaN,714.0,710.0,624.0,620.0,0.0,21926.0,NaN
3,0.0,7668185,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,422.0,93.0,53.0,NaN,NaN,...,N,N,NaN,709.0,705.0,514.0,510.0,0.0,240046.0,NaN
4,0.0,7355265,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,49.0,96.0,NaN,NaN,NaN,...,N,N,NaN,664.0,660.0,689.0,685.0,0.0,177022.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,56140812,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,6047.0,107.0,NaN,NaN,NaN,...,N,N,NaN,674.0,670.0,684.0,680.0,0.0,68046.0,NaN
99996,NaN,1167793,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,NaN,NaN,48.0,NaN,NaN,...,N,N,NaN,684.0,680.0,594.0,590.0,NaN,NaN,NaN
99997,0.0,141084437,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,56.0,151.0,NaN,NaN,NaN,...,N,N,NaN,699.0,695.0,699.0,695.0,0.0,313577.0,NaN
99998,0.0,43349780,https://lendingclub.com/browse/loanDetail.acti...,NaN,NaN,10404.0,31.0,67.0,71.0,NaN,...,N,N,NaN,674.0,670.0,564.0,560.0,0.0,63482.0,NaN


In [70]:
df_csv.to_csv('Risk_Assessment.csv',index=False)

In [60]:
len(col_list_final)

78

In [ ]:
col_dup = ['desc', 'earliest_cr_line', 'emp_length', 'emp_title', 'grade',
       'int_rate', 'verification_status_joint']
for i in col_dup:
    print(i)
    col_list_1.remove(i)
col_list_1

In [84]:
df_semifinal = df_acc[col_list]
df_semifinal

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_flag,debt_settlement_flag,debt_settlement_flag_date,fico_range_high,fico_range_low,last_fico_range_high,last_fico_range_low,tot_coll_amt,tot_cur_bal,verification_status_joint
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,N,N,NaN,679.0,675.0,564.0,560.0,722.0,144904.0,NaN
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,N,N,NaN,719.0,715.0,699.0,695.0,0.0,204396.0,NaN
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,...,N,N,NaN,699.0,695.0,704.0,700.0,0.0,189699.0,Not Verified
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,...,N,N,NaN,789.0,785.0,679.0,675.0,0.0,301500.0,NaN
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,N,N,NaN,699.0,695.0,704.0,700.0,0.0,331730.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,Vice President,9 years,...,N,N,NaN,709.0,705.0,724.0,720.0,0.0,28398.0,NaN
2260697,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,Program Manager,6 years,...,N,Y,Mar-2019,664.0,660.0,594.0,590.0,0.0,62426.0,NaN
2260698,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,10+ years,...,N,N,NaN,664.0,660.0,669.0,665.0,0.0,143066.0,NaN
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df_final = df_semifinal.sample(100000,ignore_index=True)
df_final

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_flag,debt_settlement_flag,debt_settlement_flag_date,fico_range_high,fico_range_low,last_fico_range_high,last_fico_range_low,tot_coll_amt,tot_cur_bal,verification_status_joint
0,13200.0,13200.0,13200.0,36 months,14.99,457.52,C,C5,Office manager,3 years,...,N,N,NaN,674.0,670.0,499.0,0.0,853.0,99687.0,NaN
1,32000.0,32000.0,31950.0,60 months,19.19,833.45,E,E3,Paramedic,3 years,...,N,N,NaN,699.0,695.0,744.0,740.0,0.0,271944.0,NaN
2,15000.0,15000.0,15000.0,36 months,7.24,464.81,A,A3,Sales,10+ years,...,N,N,NaN,739.0,735.0,764.0,760.0,0.0,94929.0,NaN
3,14000.0,14000.0,14000.0,36 months,9.75,450.10,B,B3,Realty Specialist,4 years,...,N,N,NaN,724.0,720.0,674.0,670.0,0.0,353569.0,NaN
4,27500.0,27500.0,27500.0,60 months,9.93,583.35,B,B2,NaN,NaN,...,N,N,NaN,729.0,725.0,499.0,0.0,0.0,20797.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,16000.0,16000.0,16000.0,36 months,9.16,509.99,B,B2,DJ,10+ years,...,N,N,NaN,804.0,800.0,584.0,580.0,0.0,159940.0,NaN
99996,2000.0,2000.0,2000.0,36 months,11.39,65.85,B,B3,Registered nurse,< 1 year,...,N,N,NaN,719.0,715.0,709.0,705.0,0.0,31243.0,NaN
99997,3000.0,3000.0,3000.0,36 months,7.26,92.99,A,A4,Shift Supervisor,5 years,...,N,N,NaN,699.0,695.0,789.0,785.0,270.0,2949.0,NaN
99998,5600.0,5600.0,5600.0,36 months,12.99,188.66,B,B5,EMT,6 years,...,N,N,NaN,679.0,675.0,759.0,755.0,0.0,95357.0,NaN


In [14]:
# COLUMN CHECK***********
col_name = 'policy_code'
col_name in df_final

True

In [15]:
for i in df_acc.columns:
    print(i)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
to

In [16]:
def Create_Samples(x,n):
    x = x.sample(n,random_state=11)
    return x

In [17]:
df_acc_s = df_acc.sample(100000,random_state=11)
df_acc_s.to_csv('accepted_1Lac_sample.csv')
df_acc_s

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
866123,136350931,NaN,15000.0,15000.0,14750.0,36 months,10.47,487.33,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
609928,113659610,NaN,35000.0,35000.0,35000.0,60 months,10.42,750.91,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1264691,17734541,NaN,25000.0,25000.0,24825.0,36 months,10.99,818.35,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1708489,97720461,NaN,20000.0,20000.0,20000.0,60 months,18.99,518.71,D,D3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1973717,88392754,NaN,20000.0,20000.0,20000.0,36 months,11.49,659.43,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080197,123071825,NaN,25600.0,25600.0,25600.0,60 months,12.62,577.52,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1936193,1122866,NaN,21000.0,21000.0,21000.0,60 months,19.42,549.62,E,E3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1510717,134205185,NaN,8450.0,8450.0,8450.0,36 months,8.08,265.11,A,A5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
552446,119217161,NaN,10000.0,10000.0,10000.0,36 months,10.91,326.97,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# df_acc_s[df_acc_s.grade == 'E']

In [19]:
# df_acc[df_acc['grade'] == 'E']

In [20]:
# df_final = 

In [21]:
df_acc_s.iloc[0:2]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
866123,136350931,NaN,15000.0,15000.0,14750.0,36 months,10.47,487.33,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
609928,113659610,NaN,35000.0,35000.0,35000.0,60 months,10.42,750.91,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_acc.columns.get_loc('loan_status')

16

## 3. EXPLORATORY DATA ANALYSIS

In [23]:
# summarizing, visualizing and getting deeply acquainted with the important traits
# of a data set.

In [24]:
df_acc_s.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,9.999800e+04,99932.000000,99997.000000,99998.000000,...,476.0,476.000000,476.0,476.000000,380.000000,476.000000,476.000000,1532.000000,1532.000000,1532.000000
mean,NaN,15014.840797,15009.830697,14992.696191,13.099164,445.037031,7.790868e+04,18.781950,0.306359,698.562721,...,3.0,158.317983,3.0,13.644958,452.163000,12102.047563,197.640966,4983.275313,47.638832,13.397520
std,NaN,9180.119379,9178.605648,9181.304054,4.839717,266.511746,7.608370e+04,15.104143,0.855246,32.917146,...,0.0,129.381329,0.0,9.595284,362.617395,7743.670817,200.377065,3809.113731,6.782307,8.002405
min,NaN,500.000000,500.000000,0.000000,5.310000,15.670000,0.000000e+00,0.000000,0.000000,640.000000,...,3.0,5.400000,3.0,0.000000,16.200000,795.320000,0.010000,135.300000,10.690000,0.000000
25%,NaN,8000.000000,8000.000000,8000.000000,9.490000,251.400000,4.600000e+04,11.860000,0.000000,675.000000,...,3.0,62.460000,3.0,5.000000,186.157500,5750.427500,52.767500,2100.000000,45.000000,6.000000
50%,NaN,12800.000000,12800.000000,12800.000000,12.620000,378.580000,6.500000e+04,17.780000,0.000000,690.000000,...,3.0,120.220000,3.0,14.000000,350.595000,11010.285000,136.135000,4031.515000,45.000000,14.000000
75%,NaN,20000.000000,20000.000000,20000.000000,15.990000,591.480000,9.300000e+04,24.410000,0.000000,715.000000,...,3.0,217.237500,3.0,22.000000,611.265000,16808.015000,290.045000,6807.790000,50.000000,18.000000
max,NaN,40000.000000,40000.000000,40000.000000,30.990000,1628.080000,9.225000e+06,999.000000,20.000000,845.000000,...,3.0,893.050000,3.0,30.000000,2679.150000,35582.500000,1253.280000,28000.000000,85.000000,36.000000


In [25]:
df_acc_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 866123 to 1715519
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 116.0+ MB


Indeed some of the features are spelled differently but mean the same thing, for example `verified_status_joint` and `verification_status_joint`. So I remove wrong and add correct ones to the list `browse_feat`.

In [26]:
# wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
#          'mths_since_recent_loan_delinq', 'verified_status_joint']
# correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
#            'mths_since_recent_bc_dlq', 'verification_status_joint']

# browse_feat = np.setdiff1d(browse_feat, wrong)
# browse_feat = np.append(browse_feat, correct)

Finally all matching features are saved in the list `avail_feat` a new DataFrame `X` that only contains these features, is created. It is a good practice to set every newly created DataFrame as a copy in order to avoid hidden chained assignments and `SettingWithCopyWarning` further down the code.

In [27]:
# avail_feat = np.intersect1d(browse_feat, data_feat)
# X = data[avail_feat].copy()
# X.info()

# 4. Feature Engineering

Let's check the categorical features and see if any of them could be transformed to other types.

In [28]:
X = df_acc_s.copy()
X

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
866123,136350931,NaN,15000.0,15000.0,14750.0,36 months,10.47,487.33,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
609928,113659610,NaN,35000.0,35000.0,35000.0,60 months,10.42,750.91,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1264691,17734541,NaN,25000.0,25000.0,24825.0,36 months,10.99,818.35,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1708489,97720461,NaN,20000.0,20000.0,20000.0,60 months,18.99,518.71,D,D3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1973717,88392754,NaN,20000.0,20000.0,20000.0,36 months,11.49,659.43,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080197,123071825,NaN,25600.0,25600.0,25600.0,60 months,12.62,577.52,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1936193,1122866,NaN,21000.0,21000.0,21000.0,60 months,19.42,549.62,E,E3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1510717,134205185,NaN,8450.0,8450.0,8450.0,36 months,8.08,265.11,A,A5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
552446,119217161,NaN,10000.0,10000.0,10000.0,36 months,10.91,326.97,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
X.select_dtypes('object').head()

,id,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
866123,136350931,36 months,B,B2,National Sales Manager,7 years,MORTGAGE,Source Verified,Jul-2018,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
609928,113659610,60 months,B,B3,Lending Manager,10+ years,MORTGAGE,Verified,Aug-2017,Current,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1264691,17734541,36 months,B,B3,Compliance Specialist,8 years,MORTGAGE,Source Verified,Jun-2014,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1708489,97720461,60 months,D,D3,Recovery Manager,10+ years,MORTGAGE,Source Verified,Feb-2017,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1973717,88392754,36 months,B,B5,sr business system analyst,8 years,RENT,Verified,Aug-2016,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 866123 to 1715519
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 116.0+ MB


The features `earliest_cr_line` and `sec_app_earliest_cr_line` are dates and their type should be changed to `datetime`. Later they will be transformed to ordinal numeric features by the machine learning model.

In [31]:
X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
X['sec_app_earliest_cr_line'] = pd.to_datetime(X['sec_app_earliest_cr_line'], infer_datetime_format=True)

The features `emp_length` and `id` are numeric and their type should be changed to `float`. In case of `emp_length` I replace the extreme cases of "< 1 year" and "10+ years" with "0 years" and "11 years" respectively to separate these groups from the rest.

In [32]:
X.emp_length.value_counts()

10+ years    33125
2 years       9068
< 1 year      8427
3 years       7864
1 year        6581
5 years       6142
4 years       6078
6 years       4510
7 years       4143
8 years       4130
9 years       3436
Name: emp_length, dtype: int64

In [33]:
X['emp_length'] = X['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
X['emp_length'] = X['emp_length'].str.extract('(\d+)').astype('float')
# X['id'] = X['id'].astype('float')

## 4.2. Missing values

Although the machine learning model used here can deal with missing values, it is a good practice to do it yourself or ideally with a domain expert. Below is the table of columns with missing values and their ratio to the total number of rows.

In [34]:
null_val = X.isna().mean()
null_val = null_val[null_val != 0].sort_values()
null_val

disbursement_method                           0.00002
fico_range_low                                0.00002
fico_range_high                               0.00002
revol_bal                                     0.00002
initial_list_status                           0.00002
                                               ...   
payment_plan_start_date                       0.99524
hardship_end_date                             0.99524
hardship_payoff_balance_amount                0.99524
orig_projected_additional_accrued_interest    0.99620
member_id                                     1.00000
Length: 150, dtype: float64

In [35]:
X.desc.isna().mean()*100

94.33399999999999

The features `desc` and `member_id` are completely empty so I remove them.

In [36]:
X = X.drop(['desc', 'member_id'], axis=1, errors='ignore')

For categorical features `emp_title`, `verification_status_joint` the missing values should be filled with an empty string so these features are placed in the list `fill_empty`.

For some of the numeric features the missing values should be filled using the maximum value of the respective columns so these features are placed in the list `fill_max`. For example, the feature `mths_since_last_record` indicates the number of months since the last record (like bankruptcy, foreclosure, tax liens, etc.) so if missing, one should assume that no records were made and the number of months since the "last" record should be a maximum.

For the rest of the numeric features the missing values should be filled using the minimum value of the respective columns so these features are placed in the list `fill_min`. For example, the feature `emp_length` indicates the number of working years so if missing, one should assume that the borrower never worked and the number of working years should be a minimum.

In [37]:
fill_empty = ['emp_title', 'verification_status_joint']
fill_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']
fill_min = np.setdiff1d(X.columns.values, np.append(fill_empty, fill_max))

X[fill_empty] = X[fill_empty].fillna('')
X[fill_max] = X[fill_max].fillna(X[fill_max].max())
X[fill_min] = X[fill_min].fillna(X[fill_min].min())

/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/3711194354.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X[fill_min] = X[fill_min].fillna(X[fill_min].min())


## 4.3. Multicollinearity

Although highly correlated features (*multicollinearity*) aren't a problem for the machine learning models based on decision trees (as used here), these features decrease importances of each other and can make feature analysis more difficult. Therefore, I calculate feature correlations and remove the features with very high correlation coefficients before applying machine learning.

I start with numeric features and before calculating their correlations, it's a good practice to look at the number of their unique values.

In [38]:
num_feat = X.select_dtypes('number').columns.values
X[num_feat].nunique().sort_values()

policy_code             1
hardship_length         1
deferral_term           1
num_tl_120dpd_2m        3
num_tl_30dpd            4
                    ...  
tot_hi_cred_lim     76298
tot_cur_bal         80146
total_rec_int       85005
total_pymnt_inv     91911
total_pymnt         93844
Length: 113, dtype: int64

The feature `num_tl_120dpd_2m` has only one value (a constant) and can be removed. The feature `id` has a unique value for each row and should also be removed, otherwise the model will overfit.

In [39]:
X = X.drop(['num_tl_120dpd_2m', 'id'], axis=1, errors='ignore')

In [40]:
list1 = [1,2,3]
list2 = [3,4,5]

In [41]:
a = combinations(list1,2)
for i in a:
    print(i)

(1, 2)
(1, 3)
(2, 3)


For all pairs of the numeric features `comb_num_feat` I calculate their Pearson's R correlation coefficient and store it in `corr_num_feat`.

In [42]:
num_feat = X.select_dtypes('number').columns.values
comb_num_feat = np.array(list(combinations(num_feat, 2)))
corr_num_feat = np.array([])
for comb in comb_num_feat:
    corr = pearsonr(X[comb[0]], X[comb[1]])[0]
    corr_num_feat = np.append(corr_num_feat, corr)

/Users/rockys/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [43]:
print(corr_num_feat,type(corr_num_feat))
print(comb_num_feat,type(comb_num_feat))

[0.99976913 0.99915599 0.0961865  ... 0.79739132 0.75678776 0.84286063] <class 'numpy.ndarray'>
[['loan_amnt' 'funded_amnt']
 ['loan_amnt' 'funded_amnt_inv']
 ['loan_amnt' 'int_rate']
 ...
 ['settlement_amount' 'settlement_percentage']
 ['settlement_amount' 'settlement_term']
 ['settlement_percentage' 'settlement_term']] <class 'numpy.ndarray'>


The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.

In [44]:
high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]
high_corr_num

array([['loan_amnt', 'funded_amnt'],
       ['loan_amnt', 'funded_amnt_inv'],
       ['loan_amnt', 'installment'],
       ['funded_amnt', 'funded_amnt_inv'],
       ['funded_amnt', 'installment'],
       ['funded_amnt_inv', 'installment'],
       ['fico_range_low', 'fico_range_high'],
       ['open_acc', 'num_sats'],
       ['out_prncp', 'out_prncp_inv'],
       ['total_pymnt', 'total_pymnt_inv'],
       ['total_pymnt', 'total_rec_prncp'],
       ['total_pymnt_inv', 'total_rec_prncp'],
       ['recoveries', 'collection_recovery_fee'],
       ['tot_cur_bal', 'tot_hi_cred_lim'],
       ['num_actv_rev_tl', 'num_rev_tl_bal_gt_0'],
       ['sec_app_fico_range_low', 'sec_app_fico_range_high'],
       ['sec_app_open_acc', 'sec_app_num_rev_accts'],
       ['hardship_amount', 'hardship_payoff_balance_amount']],
      dtype='<U42')

The first feature (chosen arbitrarily) from each highly correlated feature pair is then removed.

In [45]:
X = X.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')

Then I print out the number of unique values for categorical features.

In [46]:
cat_feat = X.select_dtypes('object').columns.values
X[cat_feat].nunique().sort_values()

hardship_type                    1
term                             2
hardship_flag                    2
application_type                 2
initial_list_status              2
debt_settlement_flag             2
disbursement_method              2
pymnt_plan                       2
settlement_status                3
hardship_status                  3
verification_status              3
verification_status_joint        4
hardship_loan_status             5
home_ownership                   6
grade                            7
hardship_reason                  9
loan_status                      9
purpose                         14
hardship_start_date             24
payment_plan_start_date         26
hardship_end_date               26
sub_grade                       35
addr_state                      50
debt_settlement_flag_date       58
next_pymnt_d                    62
settlement_date                 66
last_credit_pull_d             119
last_pymnt_d                   130
issue_d             

The feature `url` has a unique value for each entry and should be removed to avoid overfitting. The feature `emp_title` has very large number of unique values and leads to the memory error when creating a contingency table, therefore I remove it as well.

In [47]:
X = X.drop(['url', 'emp_title'], axis=1, errors='ignore')

For all pairs of the categorical features `comb_cat_feat` I calculate the Cramer's V correlation coefficient that is expressed through the chi-square statistic $\chi^2$ of the contingency table:

$$ V = \sqrt{ \frac{ \chi^2 }{ n (\text{min}(K_1, K_2) - 1) } } $$

where $n$ is the sum of all elements in the contingency table, $K_1$ and $K_2$ are the dimensions of the contingency table. Note that Pearson's R correlation coefficient isn't applicable to categorical features and shouldn't be used.

In [48]:
a = ''
a in cat_feat

False

In [49]:
cat_feat

array(['term', 'grade', 'sub_grade', 'emp_title', 'home_ownership',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'purpose', 'title', 'zip_code', 'addr_state',
       'initial_list_status', 'last_pymnt_d', 'next_pymnt_d',
       'last_credit_pull_d', 'application_type',
       'verification_status_joint', 'hardship_flag', 'hardship_type',
       'hardship_reason', 'hardship_status', 'hardship_start_date',
       'hardship_end_date', 'payment_plan_start_date',
       'hardship_loan_status', 'disbursement_method',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date'], dtype=object)

In [50]:
cat_feat = X.select_dtypes('object').columns.values
comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
corr_cat_feat = np.array([])
for comb in comb_cat_feat:
#     table = pd.pivot_table(X, index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
    table1 = pd.crosstab(X[comb[0]],X[comb[1]])
    corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
    corr_cat_feat = np.append(corr_cat_feat, corr)

/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000

/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )
/var/folders/7h/c6w5g6f51h51f33hk5zh_dkw0000gn/T/ipykernel_1122/1747499616.py:7: RuntimeWarning: invalid value encountered in true_divide
  corr = np.sqrt(chi2_contingency(table1)[0] / (table1.values.sum() * (np.min(table1.shape) - 1) ) )


In [51]:
corr_cat_feat

array([3.84679038e-01, 3.97925528e-01, 1.07691244e-01, 9.75323050e-02,
       9.89022113e-02, 1.84698626e-01, 4.57931484e-03, 1.06196777e-01,
       2.23608624e-01, 1.19334875e-01, 5.66091905e-02, 1.52384647e-01,
       1.19474497e-01, 4.07180461e-02, 7.00494184e-02, 7.33853335e-02,
       7.16278595e-02, 2.69938771e-03,            nan, 1.12004390e-01,
       1.21049213e-01, 2.41419712e-01, 2.08006352e-01, 2.21064879e-01,
       1.25401061e-01, 1.35546403e-02, 3.09511279e-02, 2.18036251e-01,
       9.48637691e-02, 2.33098141e-01, 1.00000000e+00, 3.72888759e-02,
       1.75533401e-01, 8.88807906e-02, 9.91780721e-02, 1.35469949e-02,
       7.72318461e-02, 2.36810074e-01, 9.86403219e-02, 2.90321353e-02,
       1.36752595e-01, 6.88665121e-02, 1.14629059e-01, 6.71222644e-02,
       1.30252780e-02, 1.78253285e-02, 1.13102902e-02,            nan,
       1.28976515e-01, 1.61443560e-01, 2.15751499e-01, 2.16582151e-01,
       2.14798683e-01, 1.05217793e-01, 1.58998700e-01, 6.88772582e-02,
      

In [52]:
len(comb_cat_feat)

465

In [53]:
len(corr_cat_feat)

465

The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.

In [54]:
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

array([['grade', 'sub_grade'],
       ['pymnt_plan', 'hardship_end_date'],
       ['pymnt_plan', 'payment_plan_start_date'],
       ['purpose', 'title'],
       ['title', 'next_pymnt_d'],
       ['zip_code', 'addr_state'],
       ['application_type', 'verification_status_joint'],
       ['hardship_flag', 'hardship_status'],
       ['hardship_flag', 'hardship_start_date'],
       ['hardship_flag', 'hardship_end_date'],
       ['hardship_flag', 'payment_plan_start_date'],
       ['hardship_reason', 'debt_settlement_flag_date'],
       ['hardship_status', 'debt_settlement_flag_date'],
       ['hardship_status', 'settlement_date'],
       ['hardship_start_date', 'settlement_date']], dtype='<U25')

In [55]:
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

array([['grade', 'sub_grade'],
       ['pymnt_plan', 'hardship_end_date'],
       ['pymnt_plan', 'payment_plan_start_date'],
       ['purpose', 'title'],
       ['title', 'next_pymnt_d'],
       ['zip_code', 'addr_state'],
       ['application_type', 'verification_status_joint'],
       ['hardship_flag', 'hardship_status'],
       ['hardship_flag', 'hardship_start_date'],
       ['hardship_flag', 'hardship_end_date'],
       ['hardship_flag', 'payment_plan_start_date'],
       ['hardship_reason', 'debt_settlement_flag_date'],
       ['hardship_status', 'debt_settlement_flag_date'],
       ['hardship_status', 'settlement_date'],
       ['hardship_start_date', 'settlement_date']], dtype='<U25')

In [56]:
high_corr_cat[0:3,0:3]

array([['grade', 'sub_grade'],
       ['pymnt_plan', 'hardship_end_date'],
       ['pymnt_plan', 'payment_plan_start_date']], dtype='<U25')

In [57]:
high_corr_cat[:, 1]

array(['sub_grade', 'hardship_end_date', 'payment_plan_start_date',
       'title', 'next_pymnt_d', 'addr_state', 'verification_status_joint',
       'hardship_status', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'debt_settlement_flag_date',
       'debt_settlement_flag_date', 'settlement_date', 'settlement_date'],
      dtype='<U25')

In [58]:
X = X.drop(np.unique(high_corr_cat[:, 1]), axis=1, errors='ignore')

Note that for pairs of a numeric and a categorical feature correlation coefficients can't be interpreted in a meaningful way and therefore shouldn't be used.

## 4.4. Target feature

The target feature for this dataset is an indicator if the loan is good (1) or bad (0). To identify good loans, I use their loan statuses and print their counts below. The description for each status is provided by the Lending Club:

- Current: Loan is up to date on all outstanding payments.
- In Grace Period: Loan is past due but within the 15-day grace period.
- Late (16-30): Loan has not been current for 16 to 30 days.
- Late (31-120): Loan has not been current for 31 to 120 days.
- Fully paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.
- Default: Loan has not been current for an extended period of time.
- Charged Off: Loan for which there is no longer a reasonable expectation of further payments.

In [59]:
X['loan_status'].value_counts()

Fully Paid                                             47804
Current                                                38714
Charged Off                                            11857
Late (31-120 days)                                       957
In Grace Period                                          373
Late (16-30 days)                                        179
Does not meet the credit policy. Status:Fully Paid        90
Does not meet the credit policy. Status:Charged Off       23
Default                                                    1
Name: loan_status, dtype: int64

The loans with the statuses `Current` and `Fully Paid` are definitely considered good. The loans with the statuses `In Grace Period` can be considered good or not depending on strictness of the investor. In this project I consider them as good loans. All the other statuses are considered as bad loans. Note that this dataset is highly imbalanced with the minority class being 0.037 of the majority class.

In [60]:
y = X['loan_status'].copy()
y = y.isin(['Current', 'Fully Paid', 'In Grace Period']).astype('int')
y.value_counts()

1    86891
0    13109
Name: loan_status, dtype: int64

In [61]:
df1 = pd.DataFrame(y.value_counts())
df1

,loan_status
1,86891
0,13109


In [62]:
df1['Ratio'] = df1.loan_status/sum(df1.loan_status)

In [63]:
df1

,loan_status,Ratio
1,86891,0.86891
0,13109,0.13109


# 5. Model Building

As mentioned in the beginning, the goal of this project is to predict good loans among the high risk / high interest loans. The Lending Club has two features: 1) `grade` that assigns risk levels to loans ("A" for the lowest risk, "E" for the highest risk); 2) `int_rate` that assigns the interest rate according to the risk level (lowest rates for the grade "A", highest rates for the grade "E"). Therefore, for the modelling dataset `X_mod` I choose only the loans with the grade "E" and remove the features `grade` and `int_rate` since the latter is correlated with `grade` by design.

The modelling dataset is then split into training, validation and testing parts. The validation dataset will be used to adjust some of the hyperparameters such as number of iterations, precision/recall. Both splits are stratified to ensure similar distribution of classes and to avoid one of the classes being left out in the resulting splits. The latter could happen especially in the case of highly imbalanced datasets. The parameter `random_state=0` is added for the reproducibility of results.

In [64]:
X_mod = X[X.grade == 'E'].copy()
X_mod = X_mod.drop(['grade', 'int_rate'], axis=1, errors='ignore')
y_mod = y[X_mod.index]

X_train, X_test, y_train, y_test = train_test_split(X_mod, y_mod, stratify=y_mod, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=0)

For modelling I use [CatBoost](https://catboost.ai/) - a gradient boosting library based on decision trees. CatBoost is very efficient for datasets that contain categorical features with many categories. Instead of traditional one-hot encoding that generates a lot of features and makes gradient boosting of shallow decision tress difficult, CatBoost uses mean encoding that replaces each categorical feature with only one numerical feature. More about mean encoding can be found in [this video](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv).

Before fitting the model I transform the datasets and their targets into the CatBoost objects `Pool()`. The model is defined using the object `CatBoostClassifier()` with several parameters. In `CatBoostClassifier()` the optimal value for the `learning_rate` is calculated automatically for the chosen `iterations` but it's not always the best value. Therefore, I set it to the quite small value 0.03 to ensure good convergence. The parameter `iterations` is set to 1000 so that the model would converge even with such a small learning rate. Another parameter `early_stopping_rounds` is set to 100 (0.1 of the parameter `iterations`) to stop the training and to save time if overfitting is observed. Since the dataset is highly imbalanced, the ratio of two classes `n[0] / n[1]` is passed to the parameter `class_weights`. Using class weights here is equivalent to random oversampling. Note that the final model will be defined not by the last iteration but by the best error score on the evaluation dataset. One can see this optimal point on the CatBoost plot (only visible in the Edit mode of the notebook).

### CatboostModel

In [65]:
cat_feat_ind = (X_train.dtypes == 'object')

In [66]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3366 entries, 687505 to 1553971
Columns: 118 entries, term to settlement_term
dtypes: datetime64[ns](2), float64(97), object(19)
memory usage: 3.1+ MB


In [67]:
len(cat_feat_ind)

118

### LogisticModel

In [68]:
# importing required libraries

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [70]:
x_train = X_train.select_dtypes(exclude=['object','datetime64'])

In [71]:
x_test = X_test.select_dtypes(exclude=['object','datetime64'])

In [72]:
log_model = LogisticRegression()
log_model.fit(x_train,y_train)
y_pred = log_model.predict(x_test)
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9358288770053476
[[ 367   41]
 [  55 1033]]
              precision    recall  f1-score   support

           0       0.87      0.90      0.88       408
           1       0.96      0.95      0.96      1088

    accuracy                           0.94      1496
   macro avg       0.92      0.92      0.92      1496
weighted avg       0.94      0.94      0.94      1496



### RandomForest

In [73]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)
y_pred = rf_model.predict(x_test)
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9578877005347594
[[ 367    8]
 [  55 1066]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       375
           1       0.99      0.95      0.97      1121

    accuracy                           0.96      1496
   macro avg       0.93      0.96      0.95      1496
weighted avg       0.96      0.96      0.96      1496



Since the validation dataset was used to tune hyperparameters (the number of iterations), I predict targets for the testing dataset which the model hasn't seen yet. The metrics reported here are accuracy, precision and recall. They all have sensible values which is also confirmed by the confusion matrix shown below.

In [74]:
y_pred_test = model.predict(pool_test)

acc_test = accuracy_score(y_test, y_pred_test)
prec_test = precision_score(y_test, y_pred_test)
rec_test = recall_score(y_test, y_pred_test)
print(f'''Accuracy (test): {acc_test:.3f}
Precision (test): {prec_test:.3f}
Recall (test): {rec_test:.3f}''')

cm = confusion_matrix(y_test, y_pred_test)
ax = sns.heatmap(cm, cmap='viridis_r', annot=True, fmt='d', square=True)
ax.set_xlabel('Predicted')
ax.set_ylabel('True');

NameError: name 'model' is not defined

From the confusion matrix it is clear that the model tries to predict both classes and doesn't prefer one over the other due to their imbalance. The latter is a common mistake and if you see accuracy scores without the confusion matrix, be very skeptical about those results. If over-/undersampling isn't applied for imbalanced classes, the classifier will opt for the constant prediction in favor of the majority class. In this case the accuracy for this testing dataset will be quite high 0.909 (see below), although this classifier doesn't have any predictive power! This skewness in class predictions is very visible on the confusion matrix.

In [ ]:
y_pred_const = np.ones(y_test.size)

acc = accuracy_score(y_test, y_pred_const)
prec = precision_score(y_test, y_pred_const)
rec = recall_score(y_test, y_pred_const)
print(f'''Accuracy (constant prediction): {acc:.3f}
Precision (constant prediction): {prec:.3f}
Recall (constant prediction): {rec:.3f}''')

cm = confusion_matrix(y_test, y_pred_const)
ax = sns.heatmap(cm, cmap='viridis_r', annot=True, fmt='d', square=True)
ax.set_xlabel('Predicted')
ax.set_ylabel('True');